In [3]:
# ✅ **第一部分: 安装依赖**
!pip install transformers datasets torch scikit-learn matplotlib tqdm nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# ✅ **第二部分: 导入库**
import os
import re
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
# ✅ **第三部分: 确保 GPU 可用**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前设备: {device}")
!nvidia-smi

当前设备: cuda
Mon Feb 24 15:01:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+------------------------------------

In [6]:
# ✅ **第四部分: 加载数据集**
os.makedirs("data", exist_ok=True)
dataset = load_dataset("sem_eval_2010_task_8", download_mode="force_redownload")
df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/673k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/231k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2717 [00:00<?, ? examples/s]

In [7]:
# ✅ **第五部分: 数据预处理**
def extract_entities(sentence):
    """ 从句子中提取 <e1> 和 <e2> 之间的实体 """
    entity1 = re.search(r"<e1>(.*?)</e1>", sentence)
    entity2 = re.search(r"<e2>(.*?)</e2>", sentence)
    entity1 = entity1.group(1) if entity1 else ""
    entity2 = entity2.group(1) if entity2 else ""
    return entity1, entity2

def get_wordnet_definition(entity):
    """ 从 WordNet 获取实体的定义 """
    synsets = wordnet.synsets(entity)
    return synsets[0].definition() if synsets else ""

def enhance_sentence_with_knowledge(sentence):
    """ 用实体知识增强句子（仅使用 WordNet） """
    entity1, entity2 = extract_entities(sentence)
    entity1_info = get_wordnet_definition(entity1)
    entity2_info = get_wordnet_definition(entity2)
    enhanced_sentence = f"{sentence} [SEP] {entity1}: {entity1_info} [SEP] {entity2}: {entity2_info}"
    return enhanced_sentence

df_train["enhanced_sentence"] = df_train["sentence"].apply(enhance_sentence_with_knowledge)
df_test["enhanced_sentence"] = df_test["sentence"].apply(enhance_sentence_with_knowledge)

# 生成 关系标签到 ID 的映射
unique_relations = sorted(df_train["relation"].unique())
label2id = {label: idx for idx, label in enumerate(unique_relations)}
id2label = {idx: label for label, idx in label2id.items()}

df_train["label_id"] = df_train["relation"].map(label2id)
df_test["label_id"] = df_test["relation"].map(label2id)


In [8]:
# ✅ **第六部分: 加载 BERT 模型**
num_labels = len(label2id)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.to(device)
print(f"✅ 模型已加载到: {next(model.parameters()).device}")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

✅ 模型已加载到: cuda:0


In [9]:
# ✅ **第七部分: Tokenization**
def encode_texts(texts, tokenizer, max_length=256):
    return tokenizer(list(texts.values), padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

train_encodings = encode_texts(df_train["enhanced_sentence"], tokenizer)
test_encodings = encode_texts(df_test["enhanced_sentence"], tokenizer)


In [10]:
# ✅ **第八部分: 创建 PyTorch Dataset**
class RelationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

train_dataset = RelationDataset(train_encodings, df_train["label_id"].tolist())
test_dataset = RelationDataset(test_encodings, df_test["label_id"].tolist())

In [11]:
# ✅ **第九部分: 创建 DataLoader**
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

In [12]:
# ✅ **第十部分: 训练模型**
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(5):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: 平均损失 = {total_loss / len(train_loader):.4f}")

print("✅ 训练完成！")


Epoch 1: 平均损失 = 2.2420
Epoch 2: 平均损失 = 0.9700
Epoch 3: 平均损失 = 0.4801
Epoch 4: 平均损失 = 0.2747
Epoch 5: 平均损失 = 0.1647
✅ 训练完成！


In [13]:
# ✅ **第十一部分: 评估模型**
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("真实标签 (all_labels) 示例:", all_labels[:10])
print("预测标签 (all_preds) 示例:", all_preds[:10])
print("关系类别 (label2id.keys()):", list(label2id.keys()))

print("测试集性能:")
print(classification_report(all_labels, all_preds, target_names=[str(label) for label in label2id.keys()], zero_division=1))

print("✅ 代码执行完毕，训练和测试都已完成！")


真实标签 (all_labels) 示例: [14, 17, 11, 6, 1, 2, 16, 13, 2, 14]
预测标签 (all_preds) 示例: [14, 17, 11, 6, 1, 2, 18, 13, 2, 14]
关系类别 (label2id.keys()): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
测试集性能:
              precision    recall  f1-score   support

           0       0.93      0.96      0.94       134
           1       0.94      0.91      0.92       194
           2       0.79      0.89      0.84       162
           3       0.77      0.85      0.81       150
           4       0.92      0.86      0.89       153
           5       0.86      0.95      0.90        39
           6       0.92      0.93      0.92       291
           7       1.00      0.00      0.00         1
           8       0.89      0.86      0.88       211
           9       0.93      0.87      0.90        47
          10       0.58      0.68      0.62        22
          11       0.80      0.85      0.82       134
          12       0.63      0.84      0.72        32
          13       0.88     